# Some Demonstrations with Minikube and Containers

## Cleaning Up

First, the traces of former executions of  *minikube* are eliminated, the cluster itself (stop, delete) and further initialization steps in the cache directory to establish a clean base.

In [1]:
!minikube stop ; minikube delete ; rm -rf ~/.minikube/cache

✋  Stopping node "minikube"  ...
🛑  Powering off "minikube" via SSH ...
🛑  1 nodes stopped.
🔥  Deleting "minikube" in docker ...
🔥  Deleting container "minikube" ...
🔥  Removing /Users/lothar/.minikube/machines/minikube ...
💀  Removed all traces of the "minikube" cluster.


## Start The First Cluster

In [2]:
!KUBERNETES_VERSION=1.22.1 minikube start --kubernetes-version=${KUBERNETES_VERSION}

😄  minikube v1.23.0 on Darwin 11.5.2
✨  Automatically selected the docker driver. Other choices: hyperkit, virtualbox, ssh, podman (experimental)
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
💾  Downloading Kubernetes v1.22.1 preload ...
    > preloaded-images-k8s-v12-v1...: 515.04 MiB / 515.04 MiB  100.00% 6.08 MiB
🔥  Creating docker container (CPUs=2, Memory=1987MB) ...
🐳  Preparing Kubernetes v1.22.1 on Docker 20.10.8 ...
    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


## Checking Readiness

In [3]:
!kubectl get nodes

NAME       STATUS     ROLES                  AGE   VERSION
minikube   NotReady   control-plane,master   10s   v1.22.1


In [4]:
!sleep 10 ; kubectl get nodes

NAME       STATUS   ROLES                  AGE   VERSION
minikube   Ready    control-plane,master   20s   v1.22.1


## Start Cluster With Preloaded Cache

In [5]:
!minikube stop ; minikube delete

✋  Stopping node "minikube"  ...
🛑  Powering off "minikube" via SSH ...
🛑  1 nodes stopped.
🔥  Deleting "minikube" in docker ...
🔥  Deleting container "minikube" ...
🔥  Removing /Users/lothar/.minikube/machines/minikube ...
💀  Removed all traces of the "minikube" cluster.


In [6]:
!KUBERNETES_VERSION=1.22.1 minikube start --kubernetes-version=${KUBERNETES_VERSION}

😄  minikube v1.23.0 on Darwin 11.5.2
✨  Automatically selected the docker driver. Other choices: hyperkit, virtualbox, ssh, podman (experimental)
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🔥  Creating docker container (CPUs=2, Memory=1987MB) ...
🐳  Preparing Kubernetes v1.22.1 on Docker 20.10.8 ...
    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


Having deleted the cluster the startuo time for a new cluster with prefilled caches reduces to well under a minute.

In [7]:
!sleep 10

## Creating First Namespace And Pod

In [8]:
%%bash

cat <<EOF| kubectl apply -f -
kind: Namespace
apiVersion: v1
metadata:
  name: demo
  labels:
    name: demo
---
kind: Pod
apiVersion: v1
metadata:
  name: demo-pod
  namespace: demo
spec:
  containers:
  - name: demo-ctr
    image: nginx:stable-alpine
    resources:
      limits:
        memory: "200Mi"
        cpu: "700m"
      requests:
        memory: "200Mi"
        cpu: "700m"
EOF

namespace/demo created
pod/demo-pod created


A namespace demo with a pod demo-pod in it according to above yaml.

In [9]:
%%bash

cat <<EOF| kubectl apply -f -
kind: Namespace
apiVersion: v1
metadata:
  name: demo
  labels:
    name: demo
---
kind: Pod
apiVersion: v1
metadata:
  name: demo-pod
  namespace: demo
spec:
  containers:
  - name: demo-ctr
    image: nginx:stable-alpine
    resources:
      limits:
        memory: "200Mi"
        cpu: "700m"
      requests:
        memory: "200Mi"
        cpu: "700m"
EOF

namespace/demo unchanged
pod/demo-pod unchanged


As nemspace *demo* and the pod *demo-pod* in it have already been created by executing the exact same command before, *kubectl apply* remains idempotent.

In [10]:
!kubectl get pods --namespace demo

NAME       READY   STATUS              RESTARTS   AGE
demo-pod   0/1     ContainerCreating   0          0s


The newly created and started pod *demo-pod* is just one pod among others constituting the base kubernetes cluster. Those containers are isolated in the *kube-system* namespace.

In [11]:
!kubectl get namespaces

NAME              STATUS   AGE
default           Active   17s
demo              Active   1s
kube-node-lease   Active   20s
kube-public       Active   20s
kube-system       Active   20s


In [12]:
!kubectl get pods --namespace kube-system

NAME                               READY   STATUS              RESTARTS   AGE
coredns-78fcd69978-wjgrf           0/1     ContainerCreating   0          2s
etcd-minikube                      1/1     Running             0          13s
kube-apiserver-minikube            1/1     Running             0          18s
kube-controller-manager-minikube   1/1     Running             0          18s
kube-proxy-kcbwc                   1/1     Running             0          2s
kube-scheduler-minikube            1/1     Running             0          17s
storage-provisioner                1/1     Running             0          12s


In [13]:
%%bash

echo "systemd-cgls --no-pager ; exit" | minikube ssh

systemd-cgls --no-pager ; exit
]0;docker@minikube: ~docker@minikube:~$ systemd-cgls --no-pager ; exit
Control group /docker/b656577b3185794fb56b6d931c9bec7b99afad6399467b2f88b0af1fbf2b158a:
-.slice
├─init.scope 
│ └─1 /sbin/init
└─system.slice 
  ├─containerd.service 
  │ ├─ 200 /usr/bin/containerd
  │ ├─1602 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 9d95898cb83042c…
  │ ├─1603 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 69f972944518e3b…
  │ ├─1625 /usr/bin/containerd-shim-runc-v2 -namespace moby -id b2e90906de38417…
  │ ├─1663 /pause
  │ ├─1671 /pause
  │ ├─1681 /pause
  │ ├─1701 /usr/bin/containerd-shim-runc-v2 -namespace moby -id a5dd1bdf5a503a5…
  │ ├─1741 /pause
  │ ├─1786 /usr/bin/containerd-shim-runc-v2 -namespace moby -id b932b548a4ca4c4…
  │ ├─1812 /usr/bin/containerd-shim-runc-v2 -namespace moby -id aabddff2aca4184…
  │ ├─1831 /usr/bin/containerd-shim-runc-v2 -namespace moby -id e901992a1a8e5b6…
  │ ├─1874 /usr/bin/containerd-shim-runc-v2 -namespace moby

In [14]:
!rm -rf alpine

In [15]:
!skopeo --override-os linux copy docker://alpine:3.14.2 oci:alpine:3.14.2

Getting image source signatures
Copying blob a0d0a0d46f8b [======>---------------------------] 574.4KiB / 2.7MiB
Copying blob a0d0a0d46f8b [=============>----------------------] 1.1MiB / 2.7MiB
Copying blob a0d0a0d46f8b [=====================>--------------] 1.7MiB / 2.7MiB
Copying blob a0d0a0d46f8b [=============================>------] 2.2MiB / 2.7MiB
Copying blob a0d0a0d46f8b [==================================>-] 2.6MiB / 2.7MiB
Copying blob a0d0a0d46f8b [====================================] 2.7MiB / 2.7MiB
Copying blob a0d0a0d46f8b done  
Copying blob a0d0a0d46f8b done  
Copying config 696d33ca15 [====================================] 585.0b / 585.0b
Copying config 696d33ca15 done  
Copying config 696d33ca15 done  
Copying config 696d33ca15 done  
Writing manifest to image destination
Storing signatures


In [16]:
!tree alpine

alpine
├── blobs
│   └── sha256
│       ├── 03014f0323753134bf6399ffbe26dcd75e89c6a7429adfab392d64706649f07b
│       ├── 696d33ca1510966c426bdcc0daf05f75990d68c4eb820f615edccf7b971935e7
│       └── a0d0a0d46f8b52473982a3c466318f479767577551a53ffc9074c9fa7035982e
├── index.json
└── oci-layout

2 directories, 5 files


In [17]:
!file alpine/blobs/sha256/*

alpine/blobs/sha256/03014f0323753134bf6399ffbe26dcd75e89c6a7429adfab392d64706649f07b: JSON data
alpine/blobs/sha256/696d33ca1510966c426bdcc0daf05f75990d68c4eb820f615edccf7b971935e7: JSON data
alpine/blobs/sha256/a0d0a0d46f8b52473982a3c466318f479767577551a53ffc9074c9fa7035982e: gzip compressed data, original size modulo 2^32 5865472


In [18]:
!cat alpine/blobs/sha256/03014f0323753134bf6399ffbe26dcd75e89c6a7429adfab392d64706649f07b | jq

{
  "schemaVersion": 2,
  "config": {
    "mediaType": "application/vnd.oci.image.config.v1+json",
    "digest": "sha256:696d33ca1510966c426bdcc0daf05f75990d68c4eb820f615edccf7b971935e7",
    "size": 585
  },
  "layers": [
    {
      "mediaType": "application/vnd.oci.image.layer.v1.tar+gzip",
      "digest": "sha256:a0d0a0d46f8b52473982a3c466318f479767577551a53ffc9074c9fa7035982e",
      "size": 2814446
    }
  ]
}


In [19]:
!cat alpine/blobs/sha256/696d33ca1510966c426bdcc0daf05f75990d68c4eb820f615edccf7b971935e7 | jq

{
  "created": "2021-08-27T17:19:45.758611523Z",
  "architecture": "amd64",
  "os": "linux",
  "config": {
    "Env": [
      "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"
    ],
    "Cmd": [
      "/bin/sh"
    ]
  },
  "rootfs": {
    "type": "layers",
    "diff_ids": [
      "sha256:e2eb06d8af8218cfec8210147357a68b7e13f7c485b991c288c2d01dc228bb68"
    ]
  },
  "history": [
    {
      "created": "2021-08-27T17:19:45.553092363Z",
      "created_by": "/bin/sh -c #(nop) ADD file:aad4290d27580cc1a094ffaf98c3ca2fc5d699fe695dfb8e6e9fac20f1129450 in / "
    },
    {
      "created": "2021-08-27T17:19:45.758611523Z",
      "created_by": "/bin/sh -c #(nop)  CMD [\"/bin/sh\"]",
      "empty_layer": true
    }
  ]
}


In [20]:
!rustup self uninstall -y

info: removing rustup home
info: removing cargo home
info: removing rustup binaries
info: rustup is uninstalled


In [21]:
!rm -rf helloworld

In [22]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-apple-darwin
info: syncing channel updates for 'stable-x86_64-apple-darwin'
info: latest update on 2021-09-09, rust version 1.55.0 (c8dfcfe04 2021-09-06)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
 17.1 MiB /  17.1 MiB (100 %)   6.4 MiB/s in  2s ETA:  0s
info: downloading component 'rust-std'
 21.0 MiB /  21.0 MiB (100 %)   6.2 MiB/s in  3s ETA:  0s
info: downloading component 'rustc'
 75.9 MiB /  75.9 MiB (100 %)   6.2 MiB/s in 12s ETA:  0s
info: downloading component 'rustfmt'
info: installing component 'cargo'
info: installing component 'clippy'
info: installing component 'rust-docs'
 17.1 MiB /  17.1 MiB (100 %)   4.6 MiB/s in  2s ETA:  0s
info: installing component 'rust-std'
 21.0 MiB /  21.0 MiB (100 %)   9.7 MiB/s in  2s ETA:  0s
info: installing component 'rustc'
 75.9 MiB /  75.9 MiB (100 %)   9.6 MiB/s in  7s ETA

In [23]:
!rustup default stable

info: using existing install for 'stable-x86_64-apple-darwin'
info: default toolchain set to 'stable-x86_64-apple-darwin'

  stable-x86_64-apple-darwin unchanged - rustc 1.55.0 (c8dfcfe04 2021-09-06)



In [24]:
!rustup update

info: syncing channel updates for 'stable-x86_64-apple-darwin'
info: checking for self-updates

  stable-x86_64-apple-darwin unchanged - rustc 1.55.0 (c8dfcfe04 2021-09-06)

info: cleaning up downloads & tmp directories


In [25]:
!cargo new helloworld

     Created binary (application) `helloworld` package


In [26]:
%%bash

cd helloworld

cat >Dockerfile <<EOF
FROM docker.io/library/rust:1.55.0

WORKDIR /usr/src/helloworld

COPY . .

RUN cargo build --release

RUN cargo install --path .

CMD ["/usr/local/cargo/bin/helloworld"]
EOF

In [27]:
%%bash

cd helloworld

cat >Dockerfile.slim <<EOF
FROM docker.io/library/rust:1.55.0-alpine3.14

WORKDIR /usr/src/helloworld

COPY . .

RUN cargo build --release

RUN cargo install --path .

CMD ["/usr/local/cargo/bin/helloworld"]
EOF

In [28]:
%%bash

cd helloworld

cat >Dockerfile.slim-staged <<EOF
FROM docker.io/library/rust:1.55.0-alpine3.14

WORKDIR /usr/src/helloworld

COPY . .

RUN cargo build --release

RUN cargo install --path .

CMD ["/usr/local/cargo/bin/helloworld"]
EOF

In [29]:
!podman info

host:
  arch: amd64
  buildahVersion: 1.22.3
  cgroupControllers: []
  cgroupManager: systemd
  cgroupVersion: v2
  conmon:
    package: conmon-2.0.29-2.fc34.x86_64
    path: /usr/bin/conmon
    version: 'conmon version 2.0.29, commit: '
  cpus: 1
  distribution:
    distribution: fedora
    version: "34"
  eventLogger: journald
  hostname: localhost
  idMappings:
    gidmap:
    - container_id: 0
      host_id: 1000
      size: 1
    - container_id: 1
      host_id: 100000
      size: 65536
    uidmap:
    - container_id: 0
      host_id: 1000
      size: 1
    - container_id: 1
      host_id: 100000
      size: 65536
  kernel: 5.13.13-200.fc34.x86_64
  linkmode: dynamic
  memFree: 1698824192
  memTotal: 2061852672
  ociRuntime:
    name: crun
    package: crun-1.0-1.fc34.x86_64
    path: /usr/bin/crun
    version: |-
      crun version 1.0
      commit: 139dc6971e2f1d931af520188763e984d6cdfbf8
      spec: 1.0.0
      +SYSTEMD +SELINUX +APPARMOR +CAP +SECCOMP +EBPF +CRIU +YAJL
  os: l

In [30]:
!cd helloworld ; podman build --tag helloworld:v0.1.0 --file Dockerfile .

STEP 1/6: FROM docker.io/library/rust:1.55.0
Trying to pull docker.io/library/rust:1.55.0...
Getting image source signatures
Copying blob sha256:74ca45178655a5c9b4a372bb131e0e6a563074c81321f9178fd06bc9847f6559
Copying blob sha256:955615a668ce169f8a1443fc6b6e6215f43fe0babfb4790712a2d3171f34d366
Copying blob sha256:2756ef5f69a5190f4308619e0f446d95f5515eef4a814dbad0bcebbbbc7b25a8
Copying blob sha256:911ea9f2bd51e53a455297e0631e18a72a86d7e2c8e1807176e80f991bde5d64
Copying blob sha256:27b0a22ee906271a6ce9ddd1754fdd7d3b59078e0b57b6cc054c7ed7ac301587
Copying blob sha256:8584d51a9262f9a3a436dea09ba40fa50f85802018f9bd299eee1bf538481077
Copying blob sha256:2756ef5f69a5190f4308619e0f446d95f5515eef4a814dbad0bcebbbbc7b25a8
Copying blob sha256:74ca45178655a5c9b4a372bb131e0e6a563074c81321f9178fd06bc9847f6559
Copying blob sha256:955615a668ce169f8a1443fc6b6e6215f43fe0babfb4790712a2d3171f34d366
Copying blob sha256:911ea9f2bd51e53a455297e0631e18a72a86d7e2c8e1807176e80f991bde5d64
Copying blob sha256:27b0a

In [31]:
!podman images

REPOSITORY              TAG         IMAGE ID      CREATED       SIZE
localhost/helloworld    v0.1.0      1207419edd4b  12 hours ago  1.28 GB
docker.io/library/rust  1.55.0      378164a392dd  2 days ago    1.27 GB


In [32]:
!podman run --name helloworld --rm helloworld:v0.1.0

Hello, world!


In [33]:
!cd helloworld ; podman build --tag helloworld:v0.2.0 --file Dockerfile.slim .

STEP 1/6: FROM docker.io/library/rust:1.55.0-alpine3.14
Trying to pull docker.io/library/rust:1.55.0-alpine3.14...
Getting image source signatures
Copying blob sha256:922454686f1ed186a1f1ecf76b947b3b16125d09227ba5d5736ce77c72941ce0
Copying blob sha256:a0d0a0d46f8b52473982a3c466318f479767577551a53ffc9074c9fa7035982e
Copying blob sha256:d4ea2bb3df1724da0bb9bbf8d43f7a5b7192610627c590c2fd4b8e0770230cbf
Copying blob sha256:d4ea2bb3df1724da0bb9bbf8d43f7a5b7192610627c590c2fd4b8e0770230cbf
Copying blob sha256:a0d0a0d46f8b52473982a3c466318f479767577551a53ffc9074c9fa7035982e
Copying blob sha256:922454686f1ed186a1f1ecf76b947b3b16125d09227ba5d5736ce77c72941ce0
Copying config sha256:ac41586fb44e3aa14cee23647a0d7257d86f4a549634453ab95497bd24eff193
Writing manifest to image destination
Storing signatures
STEP 2/6: WORKDIR /usr/src/helloworld
--> 480f2a6ea05
STEP 3/6: COPY . .
--> 1860244e360
STEP 4/6: RUN cargo build --release
   Compiling helloworld v0.1.0 (/usr/src/helloworld)
    Finished release 

In [34]:
!cd helloworld ; podman build --tag helloworld:v0.3.0 --file Dockerfile.slim-staged .

STEP 1/6: FROM docker.io/library/rust:1.55.0-alpine3.14
STEP 2/6: WORKDIR /usr/src/helloworld
--> Using cache 480f2a6ea0555ca38f34d6a2949447caf1dd0adf62e6d681dce715ed793ea85f
--> 480f2a6ea05
STEP 3/6: COPY . .
--> Using cache 1860244e3609276ed56e2775f4717c23581b6b075774fb96d5c4494302ec2c8d
--> 1860244e360
STEP 4/6: RUN cargo build --release
--> Using cache 0ee3a153b7bfd68cde9043ed801a7fb16503c1866bb7a62407d0adba99ee7545
--> 0ee3a153b7b
STEP 5/6: RUN cargo install --path .
--> Using cache 941a12397d6c41507025230baf7aaaa233c99c84e6aaece58f88def0040be06b
--> 941a12397d6
STEP 6/6: CMD ["/usr/local/cargo/bin/helloworld"]
--> Using cache 62355461603acf01ed62df7828f3b59076290f5703795d74f49bef57a1744f52
COMMIT helloworld:v0.3.0
--> 62355461603
Successfully tagged localhost/helloworld:v0.3.0
Successfully tagged localhost/helloworld:v0.2.0
62355461603acf01ed62df7828f3b59076290f5703795d74f49bef57a1744f52


In [35]:
!podman images

REPOSITORY              TAG                IMAGE ID      CREATED       SIZE
localhost/helloworld    v0.3.0             62355461603a  12 hours ago  880 MB
localhost/helloworld    v0.2.0             62355461603a  12 hours ago  880 MB
localhost/helloworld    v0.1.0             1207419edd4b  12 hours ago  1.28 GB
docker.io/library/rust  1.55.0-alpine3.14  ac41586fb44e  2 days ago    868 MB
docker.io/library/rust  1.55.0             378164a392dd  2 days ago    1.27 GB


In [36]:
!podman run --name helloworld --rm helloworld:v0.3.0

Hello, world!
